In [35]:
import pandas as pd 
import sys 
import os 

file_dir = os.getcwd()
update_dir = os.path.join(file_dir, '..')
project_dir = os.path.join(update_dir, '..')

dirs = [file_dir, project_dir]
[sys.path.append(d) for d in dirs if d not in sys.path]
    
import json

from helpers.Database import Database


db = Database()
df_raw = db.get_data('bloomberg')
today = pd.to_datetime('today').date()
yesterday = (pd.to_datetime('today') - pd.DateOffset(days=1)).date()

df_raw['create_at'] = pd.to_datetime(df_raw['create_at']).dt.date
df = df_raw.copy()
df.drop_duplicates(subset=['symbol', 'broker', 'comment', 'target'], keep='first', inplace=True)

df = df.sort_values(by=['broker', 'symbol', 'date'], ascending=True)

# -------------------------------- New Listed -------------------------------- #
df['count'] = df.groupby(['broker', 'symbol'])['symbol'].transform('count')

new_listed = df[(df['count'] == 1) & (df['date'] == yesterday)]
new_listed = new_listed[['broker', 'symbol', 'target', 'comment']].reset_index(drop=True)
new_listed.sort_values('symbol',ascending=True, inplace=True)
# ---------------------------------- Changes --------------------------------- #
df['previous_target'] = df.groupby(['broker', 'symbol'])['target'].shift(1)
df['previous_comment'] = df.groupby(['broker', 'symbol'])['comment'].shift(1)
df['previous_date'] = df.groupby(['broker', 'symbol'])['date'].shift(1)
# ----------------------- Change that occured yesterday ---------------------- #
df.dropna(subset=['previous_target', 'previous_comment'], inplace=True, how ='all')
yesterday_changes = df[df['date'] == yesterday].copy()
# ------------------------------ Target Changes ------------------------------ #
target_changes = yesterday_changes[yesterday_changes['target'] != yesterday_changes['previous_target']]
target_changes = target_changes[['broker', 'target', 'previous_target', 'symbol', 'date', 'previous_date']]
target_changes['diff'] = target_changes['target'] - target_changes['previous_target']

target_changes = target_changes.astype({'target': 'str'})
target_changes.loc[:,'target'] = target_changes['target'] + ' (' + round(target_changes['diff'],2).astype(str) + ')'
target_used = target_changes[['broker', 'symbol', 'previous_target', 'target']].reset_index(drop=True)
target_used.columns = ['broker', 'symbol', 'from', 'to']

# ------------------------------ Comment Changes ----------------------------- #
comment_changes = yesterday_changes[yesterday_changes['comment'] != yesterday_changes['previous_comment']]
comment_changes = comment_changes[['broker', 'comment', 'previous_comment', 'symbol', 'date', 'previous_date']]

comment_used = comment_changes[['broker', 'symbol', 'previous_comment', 'comment']].reset_index(drop=True)
comment_used.columns = ['broker', 'symbol', 'from', 'to']

In [180]:
target_used

,broker,symbol,from,to
0,CGS International,SAWAD,38.18,39.5 (1.32)
1,Citi,JMT,16.00,13.0 (-3.0)
2,FSSIA/Finansia Syrus,JMT,19.00,17.0 (-2.0)
3,"Kasikorn Securities Co., Ltd.",THG,34.60,30.8 (-3.8)
4,Krungsri Group,BEC,4.50,4.94 (0.44)
5,Krungsri Group,JMT,26.00,18.6 (-7.4)
6,Liberator Securities Co Ltd,JMT,34.75,22.4 (-12.35)
7,Tisco Securities,DELTA,66.00,65.0 (-1.0)
8,Tisco Securities,HANA,46.00,54.0 (8.0)
9,Tisco Securities,KCE,40.00,48.25 (8.25)
